In [1]:
!wget "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
!unzip ml-25m.zip

--2024-05-09 03:01:06--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M   114MB/s    in 2.2s    

2024-05-09 03:01:09 (114 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [2]:
import pandas as pd
import numpy as np
movies_df = pd.read_csv('ml-25m/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies = movies_df['title'].values.tolist()

In [4]:
movies_df['title'] = movies_df['title'].str.lower()

In [5]:
ratings_df = pd.read_csv('ml-25m/ratings.csv', dtype={'movieId':int, 'userId':int})
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
import re
def check(val, movies):
  matched = []
  for movie in movies:
    if re.search(val, movie):
      matched.append(movie)
  return matched

In [7]:
title_to_id_moviesmap = {a:b for (a, b) in zip(movies_df['title'], movies_df['movieId'])}
id_to_title_moviesmap = {a:b for (a, b) in zip(movies_df['movieId'], movies_df['title'])}

In [8]:
def get_possible_movie_indices(search_movie, movies_map):
  possible_movies = movies_map.keys()
  for val in search_movie.split():
    possible_movies = check(val, possible_movies)

  for possible_movie in possible_movies:
    print(f'{possible_movie} : {movies_map[possible_movie]}')

  # return possible_movies
get_possible_movie_indices('scooby doo', title_to_id_moviesmap)

scooby-doo (2002) : 5419
scooby-doo 2: monsters unleashed (2004) : 7369
scooby-doo! the mystery begins (2009) : 85268
scooby-doo! curse of the lake monster (2010) : 85295
scooby-doo! frankencreepy (2014) : 117913
scooby-doo! abracadabra-doo (2010) : 118894
scooby-doo! and the witch's ghost (1999) : 119065
scooby-doo! camp scare (2010) : 122129
scooby-doo in arabian nights (1994) : 128165
scooby-doo! and the reluctant werewolf (1988) : 136245
scooby-doo on zombie island (1998) : 136247
scooby-doo! and the monster of mexico (2003) : 136249
scooby-doo! stage fright (2013) : 136251
scooby-doo! moon monster madness (2015) : 136309
chill out, scooby-doo! (2007) : 136331
scooby-doo and the ghoul school (1988) : 136333
scooby-doo and the alien invaders (2000) : 136335
scooby-doo and the cyber chase (2001) : 136337
scooby-doo! in where's my mummy? (2005) : 136339
scooby-doo! and the samurai sword (2009) : 136341
scooby-doo! and the legend of the vampire (2003) : 136343
aloha scooby-doo! (2005) 

In [9]:
id_to_title_moviesmap[54768]

'daddy day camp (2007)'

Most popular rating is 4 stars

In [12]:
def parse(vals):
  all_genres = []
  for val in vals:
    val_list = val.split('|')
    new = [x for x in val_list if x not in all_genres]
    all_genres = all_genres + new
  return all_genres

In [13]:
all_genres = parse(movies_df['genres'].tolist())
all_genres.pop(-1)

'(no genres listed)'

In [14]:
def parser(val):
  init = np.zeros(len(all_genres), dtype=int)
  val_list = val.split('|')
  if val_list[0] != '(no genres listed)':
    indices = [all_genres.index(v) for v in val_list]
    init[indices] = 1
  return init

In [15]:
movies_df['feature_vector'] = movies_df['genres'].apply(parser)

In [16]:
movies_df.head()

,movieId,title,genres,feature_vector
0,1,toy story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,jumanji (1995),Adventure|Children|Fantasy,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,grumpier old men (1995),Comedy|Romance,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,waiting to exhale (1995),Comedy|Drama|Romance,"[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,father of the bride part ii (1995),Comedy,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
movieid_to_feature_vector = {a:b for (a, b) in zip(movies_df['movieId'], movies_df['feature_vector'])}

In [18]:
import lzma
import pickle
with lzma.open('feature_vectors.xz', 'wb') as f:
  pickle.dump(movieid_to_feature_vector, f, protocol=4)

In [19]:
movies = ratings_df['movieId'].to_numpy(dtype=int)
ratings = ratings_df['rating'].to_numpy(dtype=float)
users = ratings_df['userId'].to_numpy(dtype=int)

In [ ]:
userid_to_index = {}
index_to_userid = []
movieid_to_index = {}
index_to_movieid = []
train_movie_ratings_list = []
test_movie_ratings_list = []
train_user_ratings_list = []
test_user_ratings_list = []
# movie_ratings_list = []
# user_ratings_list = []

for (user, movie, rating) in zip(users, movies, ratings):
  if movieid_to_index.get(movie) is not None:
    movie_index = movieid_to_index.get(movie)
  else:
    movie_index = len(movieid_to_index)
    index_to_movieid.append(movie)
    movieid_to_index[movie] = movie_index
    train_movie_ratings_list.append([])
    test_movie_ratings_list.append([])
    # movie_ratings_list.append([])

  if userid_to_index.get(user) is not None:
    user_index = userid_to_index.get(user)
  else:
    user_index = len(userid_to_index)
    index_to_userid.append(user)
    userid_to_index[user] = user_index
    train_user_ratings_list.append([])
    test_user_ratings_list.append([])
    # user_ratings_list.append([])

  # user_ratings_list[user_index].append((movie_index, rating))
  # movie_ratings_list[movie_index].append((user_index, rating))

  if np.random.uniform(0, 1) < 0.8:
    train_user_ratings_list[user_index].append((movie_index, rating))
    train_movie_ratings_list[movie_index].append((user_index, rating))
  else:
    test_user_ratings_list[user_index].append((movie_index, rating))
    test_movie_ratings_list[movie_index].append((user_index, rating))

In [ ]:
children_ids = []
for id, val in zip(movies_df['movieId'], movies_df['feature_vector']):
  if val[all_genres.index('Children')] == 1:
    children_ids.append(id)

In [ ]:
horror_ids = []
for id, val in zip(movies_df['movieId'], movies_df['feature_vector']):
  if val[all_genres.index('Horror')] == 1:
    horror_ids.append(id)

In [ ]:
children_indices = []
for id in children_ids:
  try:
    children_indices.append(movieid_to_index[id])
  except KeyError:
    continue

In [ ]:
horror_indices = []
for id in horror_ids:
  try:
    horror_indices.append(movieid_to_index[id])
  except KeyError:
    continue

In [ ]:
import pickle
with open('indices.pkl', 'wb') as f:
  pickle.dump((horror_indices, children_indices), f, protocol=4)